In [6]:
import tkinter as tk
from tkinter import simpledialog, messagebox
import pandas as pd
import os
import subprocess
import sys
from datetime import datetime

def get_user_inputs():
    root = tk.Tk()
    root.withdraw()  # Hide the main window

    valid_event_types = {'ESPP', '401k'}
    valid_months = {'April', 'October'}

    while True:
        data_pull_type = simpledialog.askstring("Data Pull Type", "Enter data pull type (ESPP or 401k):")
        if data_pull_type not in valid_event_types:
            messagebox.showerror("Invalid Input", "Please enter a valid data pull type (ESPP or 401k).")
            continue

        month_option = simpledialog.askstring("Month Option", "Choose month (April or October):")
        if month_option not in valid_months:
            messagebox.showerror("Invalid Input", "Please enter a valid month (April or October).")
            continue

        year = simpledialog.askinteger("Year", "Enter the year:")
        current_year = datetime.now().year
        if year and 1900 <= year <= current_year + 1:
            break
        else:
            messagebox.showerror("Invalid Input", "Please enter a valid year.")
    
    root.destroy()
    return data_pull_type, month_option, year

def calculate_date_range(month_option, year):
    if month_option == "April":
        start_date = f"{year - 1}-10-16"
        end_date = f"{year}-04-15"
    else:  # October
        start_date = f"{year}-04-16"
        end_date = f"{year}-10-15"
    return start_date, end_date

def filter_attendees(data_pull_type, start_date, end_date, attendee_db):
    filtered_attendees = attendee_db[
        (attendee_db['Event Type'] == data_pull_type) &
        (attendee_db['Event Date'] >= start_date) &
        (attendee_db['Event Date'] <= end_date)
    ][['Participant Id', 'Event Type', 'Event Date']]
    return filtered_attendees

def create_unique_filename(base_filename):
    version = 1
    filename, file_extension = os.path.splitext(base_filename)
    new_filename = base_filename

    while os.path.exists(new_filename):
        new_filename = f"{filename}_{version}{file_extension}"
        version += 1

    return new_filename

def save_and_open_csv(data, base_filename='Filtered_Attendee_Data.csv'):
    unique_filename = create_unique_filename(base_filename)
    data.to_csv(unique_filename, index=False)
    try:
        os.startfile(unique_filename)  # For Windows
    except AttributeError:
        if sys.platform == 'darwin':  # For MacOS
            subprocess.call(['open', unique_filename])
        else:  # For Linux
            subprocess.call(['xdg-open', unique_filename])

def main():
    data_pull_type, month_option, year = get_user_inputs()
    start_date, end_date = calculate_date_range(month_option, year)

    attendee_db_path = "Attendee-Database.csv"
    if not os.path.exists(attendee_db_path):
        messagebox.showerror("File Not Found", "Attendee database file not found. Please check the file path.")
        return

    attendee_db = pd.read_csv(attendee_db_path)
    attendees = filter_attendees(data_pull_type, start_date, end_date, attendee_db)
    save_and_open_csv(attendees)

if __name__ == "__main__":
    main()
